# Template Notebook for NSGA II Multi-Objective-Optimization exploring the Prompt Embedding Space
Notebook Version: 0.7.0 (07/07/2025)
* update pip install to include imaging

## Google Colab Setup

In [ ]:
# Google Colab: Execute this to install packages and setup drive
# May need to restart the runtime after this step.
# May replace imaging with sound if you want to use the sound subpackage.
!pip install "evolutionary[prompt_embedding,imaging] @ git+https://git@github.com/malthee/evolutionary-diffusion.git"

In [ ]:
# Mount drive to save results
from google.colab import drive
import evolutionary_imaging.processing as ip
import evolutionary_prompt_embedding.tensorboard_embed_visualizer as ev
drive.mount("/content/drive")
base_path = "/content/drive/MyDrive/evolutionary/"
ip.RESULTS_FOLDER = base_path + ip.RESULTS_FOLDER
ev.DEFAULT_OUTPUT_FOLDER = base_path + "vis"
save_run_path = base_path + "saved_runs"

In [ ]:
# Check if GPU is available
import torch
print(torch.cuda.is_available())

## Project Setup

In [ ]:
from evolutionary.plotting import plot_fitness_statistics, plot_time_statistics
import evolutionary_imaging.processing as ip
from diffusers.utils import logging
from evolutionary_imaging.processing import create_animation_from_generations, create_generation_radar_chart_grid, save_images_from_generation
import torch
import os

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To remove warning of libraries using tokenizers
# Change the results folder for images and embedding visualization if you want to
# ip.RESULTS_FOLDER = "choose_your_destination"
# ev.DEFAULT_OUTPUT_FOLDER = "choose_your_destination"
# save_run_path = "saved_runs"
use_visualizer = True # Set to False if you don't want to use the TensorboardEmbedVisualizer
save_images = True # Set to False if you don't want to save images
print(torch.random.get_rng_state()) # Check torch random state, used across all libraries. Caution setting fixed seeds as it affects not only generation but also variation.

In [ ]:
from evolutionary_prompt_embedding.tensorboard_embed_visualizer import TensorboardEmbedVisualizer
from evolutionary_prompt_embedding.argument_types import PooledPromptEmbedData
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary_prompt_embedding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.evolutionary_selectors import TournamentSelector
from evolutionary.algorithms.nsga_ii import NSGA_II, NSGATournamentSelector
from evolutionary_imaging.evaluators import AestheticsImageEvaluator, CLIPScoreEvaluator, MultiCLIPIQAEvaluator, \
    SingleCLIPIQAEvaluator
from evolutionary.evaluators import MultiObjectiveEvaluator
from evolutionary_prompt_embedding.value_ranges import SDXLTurboEmbeddingRange, SDXLTurboPooledEmbeddingRange

visualizer = TensorboardEmbedVisualizer[PooledPromptEmbedData, [str, str, str]](["Index", "Generation", "Fitness"]) # Could add more to split fitnesses

def post_sort_callback(g, a):
    image_paths = None
    if save_images:
        image_paths = save_images_from_generation(a.population, g)
    if use_visualizer:
        for i, s in enumerate(a.population):
            fitness_val = s.fitness if not isinstance(s.fitness, list) else sum(s.fitness)/len(s.fitness)
            visualizer.add_embedding(s.arguments, [str(i), str(g), f"{fitness_val:.3f}"], image_paths[i] if image_paths else None)

population_size = 50
num_generations = 50
batch_size = 1
elitism = 1
inference_steps = 4
metrics = ("scary", "beautiful", "quality")

embedding_range = SDXLTurboEmbeddingRange()
pooled_embedding_range = SDXLTurboPooledEmbeddingRange()
creator = SDXLPromptEmbeddingImageCreator(batch_size=batch_size, inference_steps=inference_steps)
evaluator = MultiCLIPIQAEvaluator(metrics=metrics)
crossover = PooledArithmeticCrossover(interpolation_weight=0.5, interpolation_weight_pooled=0.5)
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=2, 
                                                     clamp_range=(embedding_range.minimum, embedding_range.maximum)) 
mutation_arguments_pooled = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.4, 
                                                            clamp_range=(pooled_embedding_range.minimum, pooled_embedding_range.maximum))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments_pooled)
selector = NSGATournamentSelector()

# Prepare initial arguments
#init_embed = creator.arguments_from_prompt(prompt) # with prompt
#init_args = [init_embed for _ in range(population_size)]
init_args = [PooledPromptEmbedData(embedding_range.random_tensor_in_range(), pooled_embedding_range.random_tensor_in_range()) 
             for _ in range(population_size)] # Random embeddings

nsga = NSGA_II(
    num_generations=num_generations,
    population_size=population_size,
    solution_creator=creator,
    selector=selector,
    crossover=crossover,
    mutator=mutator,
    evaluator=evaluator,
    elitism_count=elitism,
    initial_arguments=init_args,
    post_non_dominated_sort_callback=post_sort_callback
)

In [ ]:
best_solution = nsga.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
print(best_solution.fitness)
make_image_grid(best_solution.result.images, 1, batch_size)

## Visualize Embeddings with the Tensorboard Embedding Projector

In [ ]:
# This will save the embeddings and the metadata to your disk
visualizer.generate_visualization(
    sprite_single_image_dim=(80, 80),
    #filter_predicate=lambda e, l, i: int(l[0]) < 30 # When too large, use this as a filter
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={visualizer.output_folder}

## Create a video from the generational progress showing the top images

In [ ]:
for gen in range(nsga.completed_generations):
    create_generation_radar_chart_grid(gen, tuple(m if isinstance(m, str) else m[0] for m in metrics), 
                                       max_images=4, 
                                       label_padding=12,
                                       max_value=1.0 # Depends on the objectives set above
                                       )

In [ ]:
video_loc = create_animation_from_generations(nsga.completed_generations)
print(video_loc)

## Plot fitness statistics (Separately)

In [ ]:
stats = nsga.statistics

In [ ]:
plot_fitness_statistics(nsga.completed_generations, stats.best_fitness, stats.worst_fitness, stats.avg_fitness, title=metrics[0], multi_objective_plot_index=0)

In [ ]:
plot_fitness_statistics(nsga.completed_generations, stats.best_fitness, stats.worst_fitness, stats.avg_fitness, title=metrics[1], multi_objective_plot_index=1)

## Plot fitness statistics (Together)

In [ ]:
plot_fitness_statistics(nsga.completed_generations, avg_fitness=stats.best_fitness, labels=[m if isinstance(m, str) else " ".join(m) for m in metrics])

In [ ]:
plot_fitness_statistics(nsga.completed_generations, avg_fitness=stats.avg_fitness, labels=[m if isinstance(m, str) else " ".join(m) for m in metrics])

## Plot time statistics

In [ ]:
plot_time_statistics(stats.evaluation_time, stats.creation_time)

## Save notebook and components

In [ ]:
!jupyter nbconvert --to html nsga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
output_file = os.path.join("saved_runs", f"nsga_iqa_v0_7_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")
with open(output_file, "wb") as f:
    pickle.dump(nsga, f)
print(f"Run saved to {output_file}")

### Load the run from disk 
Notebook and library versions should match with the saved run

In [ ]:
import pickle
import os

with open(os.path.join("saved_runs", "insert_filename"), "rb") as f:
    nsga = pickle.load(f)

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary_imaging.processing as ip
from PIL import Image

num_generations = nsga.completed_generations  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join("results", f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary_imaging.processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)